In [21]:
#from google.colab import drive
#
#drive.mount('/content/gdrive')

# Модули

In [49]:
import numpy as np
import os
import matplotlib.pyplot as plt
from imutils import paths

from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import classification_report

# Глобални променливи

In [23]:
init_learn_rate = 1e-4
batch_size = 32
epochs = 20

# Вчитување на податочното множество

In [24]:
images = []
labels = []
imagePaths = list(paths.list_images("/content/drive/My Drive/Dataset"))
i=0
for imagePath in imagePaths:
    image = load_img(imagePath, target_size=(224,224))
    image = img_to_array(image)
    image = preprocess_input(image)
    images.append(image)
    
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)
    i=i+1
    print("Вкупно обработени слики:",i)

Вкупно обработени слики: 1
Вкупно обработени слики: 2
Вкупно обработени слики: 3
Вкупно обработени слики: 4
Вкупно обработени слики: 5
Вкупно обработени слики: 6
Вкупно обработени слики: 7
Вкупно обработени слики: 8
Вкупно обработени слики: 9
Вкупно обработени слики: 10
Вкупно обработени слики: 11
Вкупно обработени слики: 12
Вкупно обработени слики: 13
Вкупно обработени слики: 14
Вкупно обработени слики: 15
Вкупно обработени слики: 16
Вкупно обработени слики: 17
Вкупно обработени слики: 18
Вкупно обработени слики: 19
Вкупно обработени слики: 20
Вкупно обработени слики: 21
Вкупно обработени слики: 22
Вкупно обработени слики: 23
Вкупно обработени слики: 24
Вкупно обработени слики: 25
Вкупно обработени слики: 26
Вкупно обработени слики: 27
Вкупно обработени слики: 28
Вкупно обработени слики: 29
Вкупно обработени слики: 30
Вкупно обработени слики: 31
Вкупно обработени слики: 32
Вкупно обработени слики: 33
Вкупно обработени слики: 34
Вкупно обработени слики: 35
Вкупно обработени слики: 36
В

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Вкупно обработени слики: 443
Вкупно обработени слики: 444
Вкупно обработени слики: 445
Вкупно обработени слики: 446
Вкупно обработени слики: 447
Вкупно обработени слики: 448
Вкупно обработени слики: 449
Вкупно обработени слики: 450
Вкупно обработени слики: 451
Вкупно обработени слики: 452
Вкупно обработени слики: 453
Вкупно обработени слики: 454
Вкупно обработени слики: 455
Вкупно обработени слики: 456
Вкупно обработени слики: 457
Вкупно обработени слики: 458
Вкупно обработени слики: 459
Вкупно обработени слики: 460
Вкупно обработени слики: 461
Вкупно обработени слики: 462
Вкупно обработени слики: 463
Вкупно обработени слики: 464
Вкупно обработени слики: 465
Вкупно обработени слики: 466
Вкупно обработени слики: 467
Вкупно обработени слики: 468
Вкупно обработени слики: 469
Вкупно обработени слики: 470
Вкупно обработени слики: 471
Вкупно обработени слики: 472
Вкупно обработени слики: 473
Вкупно обработени слики: 474
Вкупно обработени слики: 475
Вкупно обработени слики: 476
Вкупно обработ

In [25]:
# Конвертирање на податоците во NumPy низи
data = np.array(images, dtype="float32")
labels = np.array(labels)


In [26]:
# Енкодирање на целниот атрибут (ознаките на сликите) преку One-Hot кодирачка шема
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)


# Делење на податочното множество на тренирачко и тестирачко
### 75% од податоците се користат за тренирање, а 25% за тестирање

In [27]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.25, stratify=labels, random_state=42)

# Генератор за зголемување на тренирачкото податочно множество
### Користи техники како ротација, скалирање, поместување на веќе постоечките слики за тренирање со цел да обезбеди повеќе слики од кои моделот ќе учи

In [28]:
img_gen = ImageDataGenerator(rotation_range=20, zoom_range=0.15, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15, horizontal_flip=True, fill_mode="nearest")

# Креирање на моделот
### Се користи техниката наречена transfer learning каде се користи основата на веќе истренирана конволуциска невронска мрежа на огромно податочно множество - MobileNetV2, додека последниот слој го дефинираме самите и го тренираме на нашето податочно множество со што се специфицира знаењето на моделот. Кога се тренира додадениот слој на мрежата, останатите слоеви се „замрзнуваат“ за да не се менуваат тежините.

In [30]:
base = MobileNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))


In [31]:
top_layer = base.output
top_layer = AveragePooling2D(pool_size=(7, 7))(top_layer)
top_layer = Flatten(name="flatten")(top_layer)
top_layer = Dense(128, activation="relu")(top_layer)
top_layer = Dropout(0.5)(top_layer)
top_layer = Dense(2, activation="softmax")(top_layer)

In [32]:
cnn_model = Model(inputs=base.input, outputs=top_layer)
for layer in base.layers:
	layer.trainable = False
opt = Adam(lr=init_learn_rate, decay=init_learn_rate / epochs)
cnn_model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# Тренирање на моделот

In [36]:
res = cnn_model.fit(
	img_gen.flow(X_train, y_train, batch_size=batch_size),
	steps_per_epoch=len(X_train) // batch_size,
	validation_data=(X_test, y_test),
	validation_steps=len(X_test) // batch_size,
	epochs=epochs)

Epoch 1/20
90/90 [==============================] - 39s 431ms/step - loss: 0.3476 - accuracy: 0.8443 - val_loss: 0.1146 - val_accuracy: 0.9688
Epoch 2/20
90/90 [==============================] - 37s 408ms/step - loss: 0.1362 - accuracy: 0.9534 - val_loss: 0.0721 - val_accuracy: 0.9813
Epoch 3/20
90/90 [==============================] - 37s 406ms/step - loss: 0.0879 - accuracy: 0.9681 - val_loss: 0.0572 - val_accuracy: 0.9834
Epoch 4/20
90/90 [==============================] - 36s 405ms/step - loss: 0.0777 - accuracy: 0.9698 - val_loss: 0.0527 - val_accuracy: 0.9844
Epoch 5/20
90/90 [==============================] - 37s 413ms/step - loss: 0.0641 - accuracy: 0.9804 - val_loss: 0.0449 - val_accuracy: 0.9865
Epoch 6/20
90/90 [==============================] - 37s 409ms/step - loss: 0.0620 - accuracy: 0.9828 - val_loss: 0.0446 - val_accuracy: 0.9865
Epoch 7/20
90/90 [==============================] - 36s 404ms/step - loss: 0.0544 - accuracy: 0.9807 - val_loss: 0.0438 - val_accuracy: 0.9865

# Евалуација на моделот

In [42]:
predicted_values = cnn_model.predict(X_test, batch_size=batch_size)

In [43]:
predicted_values = np.argmax(predicted_values, axis=1)

In [44]:
predicted_values.shape

(962,)

In [47]:
y_test = np.argmax(y_test, axis=1)

In [50]:
print(classification_report(y_test, predicted_values, target_names=lb.classes_))

              precision    recall  f1-score   support

        mask       0.99      1.00      0.99       479
     no_mask       1.00      0.99      0.99       483

    accuracy                           0.99       962
   macro avg       0.99      0.99      0.99       962
weighted avg       0.99      0.99      0.99       962



# Зачувување на моделот

In [53]:
cnn_model.save("/content/drive/My Drive/MaskDetectionModel.model",save_format="h5")